# Demo "Relaxed Releases with DB-Workloadtests": Workload Capture

## Disclaimer
THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED
TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR
PARTICULAR PURPOSE. 

CODE IS WRITTEN FOR DEMO PURPOSES 

(C) Martin Guth and others, 2020

## Prerequisites
* SQL Server 2008 or newer (tested with SQL Server 2019 on Linux resp Docker)
* [StackOverflow-Database](https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/) (tested with StackOverflow2010 aka small edition)
* [SQL Query Stress](https://ci.appveyor.com/api/projects/ErikEJ/SqlQueryStress/artifacts/SqlQueryStress.zip?branch=master) for workload generation
* [sp_whoIsActive](http://whoisactive.com/downloads/) to monitor activity in the database
* [WorkloadTools](https://github.com/spaghettidba/WorkloadTools/releases) to capture the workload
* [DB Browser for SQLite](https://sqlitebrowser.org/dl/) to inspect the SQLite-File resulting from the capture
* [dbatools](https://dbatools.io/download/) to facilitate database access prior to running the capture

## Explanation of the PowerShell-Script for Capture
Vorbereitend wird die aktuelle Zeit in einer Datei gespeichert und in der Datenbank eine Marked-Transaction durchgeführt.

Diese ermöglicht in Datenbanken mit viel Aktivität und Wiederherstellungsmodel "Vollständig" bzw. "Massenprotokolliert" zusammen mit Logbackups den exakten Point-in-Time-Restore bis zum Anfangszeitpunkt der Aufzeichnung.

Mit dem Aufruf von SqlWorkload.exe wird schließlich der Workload gestartet. Die Konfiguration wird aus der Datei capture.json verwendet.


In [1]:
$workdir = 'C:\SQL\workloadtests\Demo\en'
cd $workdir
Get-Content .\RecordWorkload.ps1

Import-Module dbatools;
$workingdir = "C:\SQL\workloadtests\Demo";
Set-Location $workingdir;

# Save SQL Authentication for the script to avoid prompts (CAUTION...insecure...only used for demo purpose)
$password = ConvertTo-SecureString 'P4$$w0rd!' -AsPlainText -Force
$credential = New-Object System.Management.Automation.PSCredential ('sa', $password)

# Save start time and generate a marked transaction
Get-Date -Format HH:mm:ss.fff > "$(Get-Date -Format yyyyMMdd_HHMM)_record.start";
$sql = "BEGIN TRANSACTION CaptureStart
   WITH MARK 'CaptureStart';
SELECT GETDATE();
COMMIT TRANSACTION CaptureStart;";
Invoke-Dbaquery -SqlInstance localhost -SqlCredential $credential -Database StackOverflow2010 -Query $sql;

# drop an existing SQLite Database to avoid appending to an existing file
If (Test-Path("$workingdir\StackOverflow.sqlite"))
{
    Remove-Item $workingdir\StackOverflow.sqlite;
}
# start the workload
& "C:\Program Files\WorkloadTools\SqlWorkload.exe" --file "$workingdir\capture.jso

Overview regarding the configuration file for capture:
- Capturing using Extended Events
- Database Connection to localhost using username and password
- Record Activity just from the database StackOverflow2010
- Write results to a SQLite-File

Hint: Change the output path for the SQLite-File to run it on your own system.


In [3]:
Get-Content $workdir\capture.json

{
    "Controller": {
        "Listener":
        {
            "__type": "ExtendedEventsWorkloadListener",
            "ConnectionInfo":
            {
                "ServerName": ".",
                "UserName": "sa",
                "Password": "P4$$w0rd!"
            },
            "DatabaseFilter" : "StackOverflow2010"
        },
        "Consumers":
        [
            {
                "__type": "WorkloadFileWriterConsumer",
                "OutputFile": "C:\\SQL\\workloadtests\\Demo\\StackOverflow_2010.sqlite" 
            }
        ]
    }
}


## Run the Capture
Now it's time to start the script for workload capture on the database StackOverflow2010.
IMPORTANT: Run the workload simulation (refer to Notebook 1_Simulate Activity) right after the capture start.
Hint: To stop capturing the workload simply stop the execution (hit the stop-button or press Strg + C in a Powershell window)

In [ ]:
.\RecordWorkload.ps1

In [ ]:
Import-Module dbatools;
# Save SQL Authentication for the script to avoid prompts (CAUTION...insecure...only used for demo purpose)
$password = ConvertTo-SecureString 'P4$$w0rd!' -AsPlainText -Force
$credential = New-Object System.Management.Automation.PSCredential ('sa', $password)
$sql = 'sp_whoIsActive'

Invoke-Dbaquery -SqlInstance . -SqlCredential $credential -Database StackOverflow2010 -Query $sql | Out-Host -FormatList;
